In [1]:
import numpy as np 
import pandas as pd 

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [2]:
df1 = pd.read_csv('/kaggle/input/luflow-network-intrusion-detection-data-set/2022/06/2022.06.12/2022.06.12.csv')
df2 = pd.read_csv('/kaggle/input/luflow-network-intrusion-detection-data-set/2022/06/2022.06.13/2022.06.13.csv')
df3 = pd.read_csv('/kaggle/input/luflow-network-intrusion-detection-data-set/2022/06/2022.06.14/2022.06.14.csv')

In [3]:
df_dataset = pd.concat([df1, df2, df3])
df_dataset.reset_index(drop=True, inplace=True)
len(df_dataset.columns)

16

In [4]:
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068376 entries, 0 to 1068375
Data columns (total 16 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   avg_ipt        1068376 non-null  float64
 1   bytes_in       1068376 non-null  int64  
 2   bytes_out      1068376 non-null  int64  
 3   dest_ip        1068376 non-null  int64  
 4   dest_port      964168 non-null   float64
 5   entropy        1068376 non-null  float64
 6   num_pkts_out   1068376 non-null  int64  
 7   num_pkts_in    1068376 non-null  int64  
 8   proto          1068376 non-null  int64  
 9   src_ip         1068376 non-null  int64  
 10  src_port       964168 non-null   float64
 11  time_end       1068376 non-null  int64  
 12  time_start     1068376 non-null  int64  
 13  total_entropy  1068376 non-null  float64
 14  label          1068376 non-null  object 
 15  duration       1068376 non-null  float64
dtypes: float64(6), int64(9), object(1)
memory usage: 130.4

In [5]:
df_dataset = df_dataset.drop(['num_pkts_out', 'num_pkts_in'], axis=1)

In [6]:
df_dataset.columns

Index(['avg_ipt', 'bytes_in', 'bytes_out', 'dest_ip', 'dest_port', 'entropy',
       'proto', 'src_ip', 'src_port', 'time_end', 'time_start',
       'total_entropy', 'label', 'duration'],
      dtype='object')

In [7]:
df_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)

In [8]:
df_dataset.dropna(inplace=True)

In [9]:
print(df_dataset.duplicated().sum())

4447


In [10]:
df_dataset.drop_duplicates(inplace = True)

In [11]:
print(df_dataset.duplicated().sum())

0


In [12]:
df_dataset.drop_duplicates(inplace = True)

In [13]:
print(df_dataset.duplicated().sum())

0


In [14]:
df_dataset["label"].value_counts()

label
benign       516220
outlier      365385
malicious     78116
Name: count, dtype: int64

In [15]:
old_value = 'outlier'
new_value = 0
df_dataset['label'] = df_dataset['label'].replace(old_value, new_value)

In [16]:
old_value = 'malicious'
new_value = 1
df_dataset['label'] = df_dataset['label'].replace(old_value, new_value)

In [17]:
old_value = 'benign'
new_value = 2
df_dataset['label'] = df_dataset['label'].replace(old_value, new_value)

/tmp/ipykernel_17/3502624556.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_dataset['label'] = df_dataset['label'].replace(old_value, new_value)


In [18]:
df_dataset["label"].value_counts()

label
2    516220
0    365385
1     78116
Name: count, dtype: int64

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df_dataset.drop(['label', 'dest_ip', 'src_ip', 'dest_port', 'src_port', 'time_start', 'time_end'], axis=1).values
y = df_dataset['label'].values

scaler = StandardScaler()
X = scaler.fit_transform(X)

y = torch.tensor(y, dtype=torch.long)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

class SimpleLinearModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(SimpleLinearModel, self).__init__()
        self.linear1 = nn.Linear(input_size, 128)
        self.linear2 = nn.Linear(128, 64)
        self.linear3 = nn.Linear(64, 32)
        self.linear4 = nn.Linear(32, output_size)
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.linear3(x)
        x = self.linear4(x)
        return x

input_size = X_train.shape[1] 
output_size = 3 
learning_rate = 0.001
epochs = 100

model = SimpleLinearModel(input_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 2 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).sum().item() / len(y_test)
    print(f'Accuracy on test set: {accuracy * 100:.2f}%')


/tmp/ipykernel_17/323831337.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.long)
/tmp/ipykernel_17/323831337.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.tensor(y_test, dtype=torch.long)


Epoch [2/100], Loss: 1.0198
Epoch [4/100], Loss: 0.9586
Epoch [6/100], Loss: 0.9031
Epoch [8/100], Loss: 0.8522
Epoch [10/100], Loss: 0.8059
Epoch [12/100], Loss: 0.7653
Epoch [14/100], Loss: 0.7315
Epoch [16/100], Loss: 0.7048
Epoch [18/100], Loss: 0.6834
Epoch [20/100], Loss: 0.6646
Epoch [22/100], Loss: 0.6479
Epoch [24/100], Loss: 0.6347
Epoch [26/100], Loss: 0.6247
Epoch [28/100], Loss: 0.6159
Epoch [30/100], Loss: 0.6082
Epoch [32/100], Loss: 0.6018
Epoch [34/100], Loss: 0.5954
Epoch [36/100], Loss: 0.5895
Epoch [38/100], Loss: 0.5849
Epoch [40/100], Loss: 0.5809
Epoch [42/100], Loss: 0.5779
Epoch [44/100], Loss: 0.5756
Epoch [46/100], Loss: 0.5732
Epoch [48/100], Loss: 0.5713
Epoch [50/100], Loss: 0.5698
Epoch [52/100], Loss: 0.5685
Epoch [54/100], Loss: 0.5674
Epoch [56/100], Loss: 0.5665
Epoch [58/100], Loss: 0.5658
Epoch [60/100], Loss: 0.5650
Epoch [62/100], Loss: 0.5642
Epoch [64/100], Loss: 0.5636
Epoch [66/100], Loss: 0.5630
Epoch [68/100], Loss: 0.5625
Epoch [70/100], Lo